In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-146660
Azure region: southcentralus
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-146660


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_cluster = 'UdacityP1'
try:
    comp_target = ComputeTarget(workspace=ws,name=compute_cluster)
    print ("Cluster exist")
except ComputeTargetException:
    comp_confg = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', max_nodes=2, min_nodes=1)
    comp_target = ComputeTarget.create(ws, compute_cluster, comp_confg)


#compute_cluster.wait_for_completion(show_output=True)

Cluster exist


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{
    '--C' : choice(0.03,0.3,3,10,30),
    '--max_iter' : choice(25,50,75,100)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 3, slack_factor = 0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory = '.',
    compute_target = comp_target,
    entry_script = 'train.py'
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig (
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=40,
    max_concurrent_runs=5,
    policy=policy,
    estimator=est
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_submission = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_submission).show()
hyperdrive_submission.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e70ca76e-e544-4260-a3c8-2bac6fac242b
Web View: https://ml.azure.com/runs/HD_e70ca76e-e544-4260-a3c8-2bac6fac242b?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-146660/workspaces/quick-starts-ws-146660&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-06-06T15:42:08.376700][API][INFO]Experiment created<END>\n""<START>[2021-06-06T15:42:09.962001][GENERATOR][INFO]Trying to sample '5' jobs from the hyperparameter space<END>\n""<START>[2021-06-06T15:42:10.139220][GENERATOR][INFO]Successfully sampled '5' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_submission.get_best_run_by_primary_metric()
print('ID : ',best_run.id)
print('Metrics : ',best_run.get_metrics())

final_model = best_run.register_model(model_name = 'hypermodel',model_path = '/outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_link = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
full_data = pd.concat([x,predict_col],axis=1)
full_data.head()



In [ ]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(full_data,test_size=0.25)
train_data.head()

In [ ]:

# Save training data to a csv file 
train_data.to_csv("bankmarket_training.csv")

In [ ]:
from azureml.core import Dataset
# Read training data from CSV File
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='training/')
final_data = Dataset.Tabular.from_delimited_files(path = [(data_store, 'training/bankmarket_training.csv')])

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=final_data,
    label_column_name='y',
    compute_target = comp_trget,
    max_concurrent_iterations = 3,
    enable_early_stopping = True,
    n_cross_validations=7)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
class_automl = exp.submit(automl_config, show_output = True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_class_automl, best_fit_model = class_automl.get_output()
print(best_fit_model)

joblib.dump(best_fit_model, 'outputs/best_fit_automl_model.joblib')
best_class_automl.register_model(model_name = 'best_automl.pkl', model_path = '/outputs/')

In [ ]:
# cluster_cleanup
comp_trget.delete()